In [2]:
!pip install einops accelerate ema-pytorch denoising-diffusion-pytorch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader

# 从 DLPM_model 中导入 1D Unet、Dataset1D 和 DLPM 噪声采样函数
from DLPM_model import Unet1D, Dataset1D, sample_dlpm_noise_like

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [4]:
# 数据预处理：读取 CSV 并构造 price_series 的对数收益率序列

import re

# 读取原始数据（路径和 unet_model.ipynb 保持一致）
train_data = pd.read_csv('trainning_data_merged.csv')

# 去掉重复逻辑的列
if 'ticker' in train_data.columns:
    train_data.drop(columns=['ticker'], inplace=True)

# 归一化日期相关特征
if 'contract_calendar_days' in train_data.columns:
    train_data['contract_calendar_days'] = train_data['contract_calendar_days'] / 365.0
if 'actual_trading_days' in train_data.columns:
    train_data['actual_trading_days'] = train_data['actual_trading_days'] / 252.0
    train_data['trading_ratio'] = train_data['actual_trading_days'] / train_data['contract_calendar_days']

# 解析 price_series 字符串为 tensor

def parse_price_series(price_list):
    """将字符串形式的价格序列解析为 float32 的 torch.tensor"""
    combined = ''.join(str(item) for item in price_list)
    numbers = re.findall(r'\d+\.\d+|\d+', combined)
    return torch.tensor([float(n) for n in numbers], dtype=torch.float32)

train_data['price_series'] = train_data['price_series'].apply(parse_price_series)

print("示例价格序列长度:", len(train_data['price_series'].iloc[0]))


示例价格序列长度: 23


In [5]:
# 将多行 price_series 转换为固定长度的对数收益率序列 (targets) 和 mask

def process_multiple_rows(price_series_column, max_length=252):
    """
    输入:
        price_series_column: pandas Series，每个元素是一个价格序列的 tensor
        max_length: 序列总长度（包含起始标记），默认为 252
    输出:
        targets: (N, max_length) 的 tensor，每行是 [start_flag, log_returns...]
        masks:   (N, max_length) 的 tensor，1 表示有效位置
    """
    n_samples = len(price_series_column)
    targets = torch.zeros(n_samples, max_length, dtype=torch.float32)
    masks = torch.zeros(n_samples, max_length, dtype=torch.float32)

    for i in range(n_samples):
        price_seq = price_series_column.iloc[i]
        log_returns = torch.diff(torch.log(price_seq))  # (T-1,)

        # 起始标记
        targets[i, 0] = 1.0
        masks[i, 0] = 1.0

        num_to_fill = min(len(log_returns), max_length - 1)
        targets[i, 1:1+num_to_fill] = log_returns[:num_to_fill]
        masks[i, 1:1+num_to_fill] = 1.0

    return targets, masks

max_seq_len = 252

targets, masks = process_multiple_rows(train_data['price_series'], max_length=max_seq_len)
print("targets 形状:", targets.shape)
print("masks 形状:", masks.shape)

# 把 targets 扩成 (N, 1, L)，作为 Unet1D 的输入/输出序列
seq_tensor = targets.unsqueeze(1).to(device)  # (N, 1, L)
print("seq_tensor 形状:", seq_tensor.shape)


targets 形状: torch.Size([65496, 252])
masks 形状: torch.Size([65496, 252])
seq_tensor 形状: torch.Size([65496, 1, 252])


In [6]:
# DLPM 余弦调度：gamma_bar, sigma_bar

def get_dlpm_cosine_schedule(T, alpha, s=0.008):
    """对应 notebook 中的 DLPM 余弦调度，返回 gamma_bar, sigma_bar（长度 T）"""
    steps = np.arange(T + 1)
    ft = np.cos(((steps / T) + s) / (1 + s) * (np.pi / 2)) ** 2
    alphas_bar = ft / ft[0]

    gamma_1_to_t = alphas_bar ** (1.0 / alpha)
    sigma_1_to_t = (1.0 - alphas_bar) ** (1.0 / alpha)

    return gamma_1_to_t[1:], sigma_1_to_t[1:]

# 超参数
T = 1000          # 总时间步
alpha = 1.0       # DLPM 尾部指数（可以改成 1.5 / 1.7 等）

# 预计算调度，并转成 torch.tensor 放在当前设备
_gamma_bar_np, _sigma_bar_np = get_dlpm_cosine_schedule(T, alpha)

gamma_bar = torch.tensor(_gamma_bar_np, dtype=torch.float32, device=device)  # (T,)
sigma_bar = torch.tensor(_sigma_bar_np, dtype=torch.float32, device=device)  # (T,)

print("gamma_bar 形状:", gamma_bar.shape)
print("sigma_bar 形状:", sigma_bar.shape)


gamma_bar 形状: torch.Size([1000])
sigma_bar 形状: torch.Size([1000])


In [7]:
# 构造 Dataset 和 DataLoader

full_dataset = Dataset1D(seq_tensor)  # 每个样本形状 (1, L)

batch_size = 64
train_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

print("数据集大小:", len(full_dataset))


数据集大小: 65496


In [8]:
# 定义最基础的 1D U-Net 模型（使用 DLPM_model 中的 Unet1D）

model = Unet1D(
    dim=64,               # 基础通道数，可以根据显存调整
    dim_mults=(1, 2, 4),  # 三层下采样/上采样
    channels=1,           # 序列是 1 通道
)

model = model.to(device)
print("模型参数量:", sum(p.numel() for p in model.parameters()) / 1e6, "M")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


模型参数量: 4.580225 M


In [ ]:
# 使用 testing_data_merged.csv 做评估：先按同样逻辑预处理，得到测试集序列

# 1. 读取测试数据
test_data = pd.read_csv('testing_data_merged.csv')

# 2. 与训练集保持一致的预处理
if 'ticker' in test_data.columns:
    test_data.drop(columns=['ticker'], inplace=True)

if 'contract_calendar_days' in test_data.columns:
    test_data['contract_calendar_days'] = test_data['contract_calendar_days'] / 365.0
if 'actual_trading_days' in test_data.columns:
    test_data['actual_trading_days'] = test_data['actual_trading_days'] / 252.0
    test_data['trading_ratio'] = test_data['actual_trading_days'] / test_data['contract_calendar_days']

# 3. 解析 price_series
test_data['price_series'] = test_data['price_series'].apply(parse_price_series)

# 4. 构造对数收益率序列
test_targets, test_masks = process_multiple_rows(test_data['price_series'], max_length=max_seq_len)
print('test_targets 形状:', test_targets.shape)
print('test_masks  形状:', test_masks.shape)

# 5. 扩成 (N_test, 1, L)
test_seq_tensor = test_targets.unsqueeze(1).to(device)
print('test_seq_tensor 形状:', test_seq_tensor.shape)

test_dataset = Dataset1D(test_seq_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print('测试集大小:', len(test_dataset))


In [9]:
# 训练循环：严格对应你 notebook 中的
# error = model(Y_t, t) - g_noise * A_half
# 的 DLPM 噪声逻辑

num_epochs = 5          # 可以先设小一点测试
print_every = 100

model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for step, x0 in enumerate(train_loader):
        # x0: (B, 1, L)
        x0 = x0.to(device)
        B = x0.size(0)

        # 1) 随机采样时间步 t ~ Uniform({0, ..., T-1})
        t = torch.randint(0, T, (B,), device=device, dtype=torch.long)  # (B,)

        # 2) 根据 t 取出对应的 gamma_t, sigma_t，并调整维度用于广播
        gamma_t = gamma_bar[t].view(B, 1, 1)  # (B,1,1)
        sigma_t = sigma_bar[t].view(B, 1, 1)  # (B,1,1)

        # 3) 生成 DLPM 重尾噪声 epsilon_t = A_half * G_t
        #    sample_dlpm_noise_like 返回 (epsilon, G_t, A_half)
        epsilon_t, g_noise, A_half = sample_dlpm_noise_like(x0, alpha=alpha)

        # 4) 按 DLPM 公式生成 Y_t
        #    Y_t = gamma_t * Y_0 + sigma_t * epsilon_t
        Y_t = gamma_t * x0 + sigma_t * epsilon_t

        # 5) 前向：模型输入 (Y_t, t)，预测 epsilon_t
        optimizer.zero_grad()

        # t 需要是 (B,) 的 long 类型
        pred_epsilon = model(Y_t, t)

        # 6) loss：和你 notebook 保持一致，预测的是 A_half * G_t
        #    error = model(Y_t, t) - g_noise * A_half
        target = epsilon_t  # 等价于 g_noise * A_half
        error = pred_epsilon - target
        mse = torch.mean(error ** 2)
        loss = torch.sqrt(mse)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (step + 1) % print_every == 0:
            avg_loss = running_loss / print_every
            print(f"Epoch [{epoch+1}/{num_epochs}] Step [{step+1}] Loss: {avg_loss:.6f}")
            running_loss = 0.0

print("训练结束！")


Epoch [1/5] Step [100] Loss: 30.726323
Epoch [1/5] Step [200] Loss: 18.552368


KeyboardInterrupt: 

In [ ]:
# 在测试集上做“反推”评估：给定 Y_0，按 DLPM 正向加噪得到 Y_t，
# 再用模型预测 epsilon_t，并反推 Y_0_hat，与真实 Y_0 比较

import matplotlib.pyplot as plt

model.eval()

all_mse = []
recon_examples = []  # 保存少量样本做可视化

# 选一个评估用的固定时间步，比如中间的 t_eval
t_eval = T // 2
print("使用评估时间步 t_eval =", t_eval)

with torch.no_grad():
    for batch_idx, x0 in enumerate(test_loader):
        # x0: (B, 1, L)
        x0 = x0.to(device)
        B = x0.size(0)

        # 1) 构造 batch 形式的时间步索引
        t_batch = torch.full((B,), t_eval, device=device, dtype=torch.long)  # (B,)

        # 2) 取出对应的 gamma_t, sigma_t
        gamma_t = gamma_bar[t_batch].view(B, 1, 1)
        sigma_t = sigma_bar[t_batch].view(B, 1, 1)

        # 3) 生成 DLPM 重尾噪声 epsilon_t = A_half * G_t
        epsilon_t, g_noise, A_half = sample_dlpm_noise_like(x0, alpha=alpha)

        # 4) 正向加噪，得到 Y_t
        Y_t = gamma_t * x0 + sigma_t * epsilon_t

        # 5) 模型预测 epsilon_t
        pred_epsilon = model(Y_t, t_batch)

        # 6) 根据公式反推 Y_0_hat:
        #    Y_t = gamma_t * Y_0 + sigma_t * epsilon_t
        # => Y_0_hat = (Y_t - sigma_t * pred_epsilon) / gamma_t
        Y0_hat = (Y_t - sigma_t * pred_epsilon) / gamma_t

        # 7) 计算该 batch 的 MSE
        batch_mse = torch.mean((Y0_hat - x0) ** 2, dim=[1, 2])  # (B,)
        all_mse.append(batch_mse.cpu())

        # 只保存前几个样本做可视化
        if batch_idx < 3:  # 取前 3 个 batch 各 1 条
            recon_examples.append((x0[0].detach().cpu(), Y0_hat[0].detach().cpu()))

# 汇总 MSE
all_mse = torch.cat(all_mse, dim=0)
print("测试集重建 MSE 均值:", all_mse.mean().item())
print("测试集重建 MSE 中位数:", all_mse.median().item())

# 可视化部分样本的原始 vs 重建对数收益率序列
num_plots = len(recon_examples)
plt.figure(figsize=(12, 4 * num_plots))

for i, (x_true, x_hat) in enumerate(recon_examples):
    plt.subplot(num_plots, 1, i + 1)
    plt.plot(x_true.squeeze().numpy(), label='True log-returns', alpha=0.8)
    plt.plot(x_hat.squeeze().numpy(), label='Reconstructed log-returns', alpha=0.8)
    plt.title(f'Sample {i+1}: True vs Reconstructed (t={t_eval})')
    plt.legend()
    plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()
